### Langchain Programming including setup in Cell 1

In [ ]:
!pip uninstall -y langchain langchain-core langchain-community langchain-experimental

Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.79
Uninstalling langchain-core-0.3.79:
  Successfully uninstalled langchain-core-0.3.79


In [ ]:
!pip install -U langchain langchain-core==1.0 langchain-community langchain-experimental langchain_google_genai

  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_experimental-0.3.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_experimental-0.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_experimental-0.3.1.post1-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_experimental-0.3.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_experimental-0.3.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_experimental-0.0.65-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_community-0.2.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_experimental-0.0.64-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is still looking at multiple versions

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

os.environ["GOOGLE_API_KEY"]=userdata.get('gemini_key')
#genai.configure(api_key=userdata.get('gemini_key'))

chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")


Prompt Templates

In [ ]:
#Prompts
from langchain_core.prompts import PromptTemplate

# Simple prompt with placeholders
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# Filling placeholders to create a prompt
filled_prompt = prompt_template.format(adjective="funny", content="robots")
print(filled_prompt)
response = chat.invoke(filled_prompt)
print(response.content)

Tell me a funny joke about robots.
Why don't robots ever get lonely?

Because they're always surrounded by their **byte**-friends!


In [ ]:
#!pip install langchain-legacy
#from langchain_experimental.agents import create_pandas_dataframe_agent

#from langchain_google_genai import GoogleGenerativeAI
#import pandas as pd

#llm=GoogleGenerativeAI(model="gemini-2.0-flash-lite")

#df = pd.read_csv('https://raw.githubusercontent.com/LearnDataSci/articles/master/Python%20Pandas%20Tutorial%20A%20Complete%20Introduction%20for%20Beginners/IMDB-Movie-Data.csv')

#agent = create_pandas_dataframe_agent(llm,df, agent_type="zero-shot-react-description",
                                      #verbose=True,return_intermediate_steps=True,
                                      #allow_dangerous_code=True)
#df.head(2)

# This functionality doesn't work anymore.
#will be moved to langchain-legacy and not recommended
# Using Agents is recommended in Langgraph from langchain 1.0 release


ImportError: cannot import name 'create_pandas_dataframe_agent' from 'langchain_experimental.agents' (/usr/local/lib/python3.12/dist-packages/langchain_experimental/agents/__init__.py)

### Using Memory and User sessions - Useful especially for Chatbots

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
model

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-lite', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x78fbd6185a30>, default_metadata=(), model_kwargs={})

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Alex and I am an AI Engineer")],
    config=config
)

In [ ]:
response.content

"Hi Alex! It's great to meet you. As an AI Engineer, you're working in a fascinating and rapidly evolving field. What kind of AI projects are you currently working on, or what areas of AI are you most interested in? I'm always eager to learn more about the cutting edge of AI development."

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Alex.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--f360fac4-af89-4abb-a9ba-18a4f3d43233-0', usage_metadata={'input_tokens': 84, 'output_tokens': 6, 'total_tokens': 90, 'input_token_details': {'cache_read': 0}})

In [ ]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I am a large language model, I do not have a name.'

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. How can I help you today?"

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John. You told me earlier! 😉'

**Using Prompt templates and LCEL with History**


*Understand and Practice on your own*

---



In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Jack")]})

AIMessage(content="Hi Jack, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--f6870a2a-9c35-4efb-8be2-cbe49fa9f85c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 19, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}})

In [ ]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Jack")],
    config=config
)

response

AIMessage(content="Hi Jack, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--994381e3-bdb9-4747-8ac6-1d8f25513be9-0', usage_metadata={'input_tokens': 21, 'output_tokens': 19, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}})

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jack! 😊'

In [ ]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Jack")],"language":"French"})
response.content

"Bonjour Jack ! Enchanté de vous rencontrer. Comment puis-je vous aider aujourd'hui ?"

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [ ]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Jack")],"language":"French"},
    config=config
)
repsonse.content

"Bonjour Jack ! Enchanté de faire votre connaissance. Comment puis-je vous aider aujourd'hui ?"

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "French"},
    config=config,
)

response.content

'Votre nom est Jack !'

### Document Loaders

In [ ]:
## Text Loader

from langchain_community.document_loaders import TextLoader

loader=TextLoader('speech.txt')
loader

In [ ]:
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [ ]:
!pip install pypdf
## Reading a PDf File
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()
docs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 13.0 MB/s eta 0:00:00


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszk

In [ ]:
type(docs[0])

langchain_core.documents.base.Document

In [ ]:
## Web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")
                     ))
                     )

In [ ]:
loader.load()

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [ ]:
## Arxiv
!pip install arxiv
!pip install pymupdf

from langchain_community.document_loaders import ArxivLoader
docs = ArxivLoader(query="1706.03762", load_max_docs=2).load()
len(docs)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=983a501b9194dba9a468379a282dbb3fd60ec37daf5f78625b5dcf88f9483302
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 46.9 MB/s eta 0:00:00


1

In [ ]:
docs

[Document(metadata={'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntr

In [ ]:
!pip install wikipedia

from langchain_community.document_loaders import WikipediaLoader
docs = WikipediaLoader(query="Agentic AI", load_max_docs=2).load()
len(docs)
print(docs)

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=b18222f97bc21dff180c5a6801180e934fda3d28f308fde9d987a6a9b21c1c76
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
[Document(metadata={'title': 'Agentic AI', 'summary': 'Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks with limited or no human intervention. The independent systems automatically respond to conditions to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence.', 'source': 'https://en.wikipedia.org/wiki/Agentic_AI'}, page_content="Agentic AI is a class of artificial intellige

### RAG with LangChain

**Embedding Techniques**

Converting text into vectors


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x78fb1cb04d10>, async_client=None, model='models/gemini-embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [ ]:
text="This is a tutorial on Gemini embedding"
query_result=embeddings.embed_query(text)
query_result

[-0.01132830698043108,
 0.01222770195454359,
 0.004136270377784967,
 -0.08865328878164291,
 -0.008162019774317741,
 0.008077823556959629,
 0.0028584832325577736,
 0.009811045601963997,
 -0.004349300172179937,
 0.003360977629199624,
 0.010015156120061874,
 -0.004345457069575787,
 -0.0012824757723137736,
 0.012536488473415375,
 0.16564269363880157,
 0.011176640167832375,
 0.005026436876505613,
 -0.004289621487259865,
 0.009985977783799171,
 -0.023122837767004967,
 -0.010932213626801968,
 -0.014704558998346329,
 0.016631944105029106,
 -0.018486635759472847,
 0.02330033667385578,
 -0.00469799293205142,
 -0.0034383225720375776,
 0.009832960553467274,
 0.01904584839940071,
 2.408282489341218e-05,
 0.008360495790839195,
 0.02026556432247162,
 0.010805092751979828,
 0.016960855573415756,
 0.015107988379895687,
 0.02117180824279785,
 0.02416655234992504,
 0.011592394672334194,
 -0.0035490572918206453,
 -0.012193175964057446,
 0.008741569705307484,
 -0.0019130123546347022,
 -0.023202823475003242

In [ ]:
len(query_result)

3072

In [ ]:
from langchain_community.document_loaders import TextLoader

loader=TextLoader('speech.txt')
docs=loader.load()
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [ ]:
# to combine multiple docs

#from langchain_community.document_loaders import TextLoader

#loader1=TextLoader('speech.txt')

## Reading a PDf File
#from langchain_community.document_loaders import PyPDFLoader
#loader2=PyPDFLoader('Extra content.pdf')

#from langchain_community.document_loaders.merge import MergedDataLoader

#loader_all = MergedDataLoader(loaders=[loader1, loader2])
#docs = loader_all.load()

In [ ]:
# Experiment with Text splitters as well using the links in python.langchain.com

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
final_documents=text_splitter.split_documents(docs)
final_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_co

In [ ]:
!pip install chromadb
## Vector Embedding And Vector StoreDB
from langchain_community.vectorstores import Chroma

db=Chroma.from_documents(final_documents,embeddings)
db



In [ ]:
### Retrieve the results from query vectorstore db
query="It will be all the easier for us to conduct ourselves as belligerents"
#query="""The primary inputs are sunlight, water, and carbon dioxide.
#It will be all the easier for us to conduct ourselves as belligerents"""

retrieved_results=db.similarity_search(query)
print(retrieved_results)

[Document(metadata={'moddate': '2025-10-25T18:35:54+05:30', 'page': 0, 'page_label': '1', 'producer': 'Microsoft® Word 2019', 'source': 'Extra content.pdf', 'creationdate': '2025-10-25T18:35:54+05:30', 'creator': 'Microsoft® Word 2019', 'author': 'Surya Surya', 'total_pages': 1}, page_content='byproduct.  \n• Summary: Photosynthesis is the process where plants use sunlight, water, and carbon \ndioxide to create their own food (glucose) and release oxygen.  \n• Inputs: The primary inputs are sunlight, water, and carbon dioxide, with chlorophyll acting as \nthe catalyst to capture light energy.  \n• Location: The process mainly happens in the leaves, specifically within organelles called \nchloroplasts.'), Document(metadata={'page': 0, 'page_label': '1', 'source': 'Extra content.pdf', 'producer': 'Microsoft® Word 2019', 'author': 'Surya Surya', 'moddate': '2025-10-25T18:35:54+05:30', 'creationdate': '2025-10-25T18:35:54+05:30', 'creator': 'Microsoft® Word 2019', 'total_pages': 1}, page_c

In [ ]:
## Retrieval Chain, Document chain
!pip install langchain-classic

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-lite', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x78fb13f781a0>, default_metadata=(), model_kwargs={})
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [ ]:
retriever=db.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x78fb1875c830>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
       

In [ ]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"why is it regarded as an irresponsible government?"})
#response=retrieval_chain.invoke({"input":"Describe Photosynthesis"})
response['answer']

'Based on the provided context, the byproduct of photosynthesis is oxygen.'

In [ ]:
response

{'input': 'Describe Photosynthesis',
 'context': [Document(metadata={'source': 'Extra content.pdf', 'page_label': '1', 'creationdate': '2025-10-25T18:35:54+05:30', 'page': 0, 'total_pages': 1, 'moddate': '2025-10-25T18:35:54+05:30', 'creator': 'Microsoft® Word 2019', 'producer': 'Microsoft® Word 2019', 'author': 'Surya Surya'}, page_content='byproduct.  \n• Summary: Photosynthesis is the process where plants use sunlight, water, and carbon \ndioxide to create their own food (glucose) and release oxygen.  \n• Inputs: The primary inputs are sunlight, water, and carbon dioxide, with chlorophyll acting as \nthe catalyst to capture light energy.  \n• Location: The process mainly happens in the leaves, specifically within organelles called \nchloroplasts.'),
  Document(metadata={'total_pages': 1, 'source': 'Extra content.pdf', 'moddate': '2025-10-25T18:35:54+05:30', 'producer': 'Microsoft® Word 2019', 'page_label': '1', 'page': 0, 'creator': 'Microsoft® Word 2019', 'author': 'Surya Surya', '